<a href="https://colab.research.google.com/github/GM-Praveen/Projects/blob/main/Smart_Document_Q%26A_Assistant_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.5.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
pandas-gbq 0.29.1 requires google-api-core<3.0.0,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
thinc 8.3

In [ ]:
!pip install langchain transformers torch sentence-transformers faiss-cpu pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 60.2 MB/s eta 0:00:00:00:0100:01
  Attempting un

In [ ]:
import gradio as gr
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
import tempfile

2025-08-06 10:46:05.926056: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754477166.126834      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754477166.196895      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
def process_pdf(pdf_file):
    global db, qa_chain

    try:
        # Get path from uploaded file (Gradio returns NamedString object with .name)
        file_path = pdf_file.name

        # Load and split document
        loader = PyPDFLoader(file_path)
        documents = loader.load()

        text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        docs = text_splitter.split_documents(documents)

        # Embeddings + FAISS
        embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
        db = FAISS.from_documents(docs, embeddings)

        # LLM pipeline
        from transformers import pipeline
        hf_pipeline = pipeline("text2text-generation", model="google/flan-t5-large", max_length=512)
        llm = HuggingFacePipeline(pipeline=hf_pipeline)

        # QA Chain
        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            retriever=db.as_retriever(search_type="similarity", search_kwargs={"k": 3}),
            return_source_documents=False
        )

        return "✅ PDF uploaded and processed successfully! You can now ask questions."

    except Exception as e:
        import traceback
        print("❌ PDF Processing Error:", traceback.format_exc())
        return f"❌ Error while processing PDF:\n{str(e)}"


In [ ]:
# Handle question asking
def ask_question(question):
    if qa_chain is None:
        return "❗ Please upload and process a PDF first."
    if not question.strip():
        return "❗ Please enter a valid question."

    prompt = (
        f"Please answer the following question based on the uploaded document. "
        f"Provide a clear, concise, and easy-to-understand summary in bullet points. "
        f"Focus on the key points only and avoid lengthy explanations.\n\n"
        f"Question: {question}"
    )

    try:
        result = qa_chain({"query": prompt})
        return result["result"]
    except Exception as e:
        return f"⚠️ Error: {e}"

In [ ]:
import gradio as gr

# Dark theme with colored accents
dark_theme = gr.themes.Base(
    primary_hue="teal",
    secondary_hue="indigo",
    neutral_hue="gray"
).set(
    button_primary_background_fill="#0d9488",        # Teal-600
    button_primary_background_fill_hover="#0f766e",  # Teal-700
    button_secondary_background_fill="#334155",      # Slate-700
    button_secondary_text_color="#f1f5f9",            # Light text
    block_border_color="#475569"                     # Slate-600 border
)

# Custom CSS for the dark layout
custom_css = """
.gradio-container {
    font-family: 'Segoe UI', sans-serif;
    background-color: #1e293b;  /* slate-800 */
    color: #e2e8f0;
}

#header-text {
    text-align: center;
    font-size: 32px;
    font-weight: bold;
    color: #f1f5f9;
    font-family: 'Georgia', serif;
}

#subheader-text {
    text-align: center;
    font-size: 17px;
    color: #cbd5e1;
    margin-bottom: 25px;
    font-style: italic;
}

.group-box {
    background-color: #334155;  /* slate-700 */
    border-radius: 12px;
    padding: 22px;
    box-shadow: 0 4px 12px rgba(0, 0, 0, 0.2);
    margin-bottom: 20px;
}

textarea, input[type="text"] {
    background-color: #1e293b !important;
    border: 1px solid #475569 !important;
    border-radius: 8px !important;
    padding: 10px !important;
    font-size: 15px !important;
    color: #e2e8f0 !important;
}
"""

with gr.Blocks(theme=dark_theme, css=custom_css) as demo:
    gr.Markdown("<div id='header-text'>📄  Smart Document Q&A Assistant</div>")
    gr.Markdown("<div id='subheader-text'>Upload a PDF and ask questions — get clean, concise answers instantly.</div>")

    with gr.Row():
        with gr.Column():
            with gr.Group(elem_classes="group-box"):
                pdf_file = gr.File(label="📁 Upload PDF", file_types=[".pdf"])
                process_btn = gr.Button("🚀 Process PDF", variant="primary")
                status_box = gr.Textbox(label="📌 Status", interactive=False)

        with gr.Column():
            with gr.Group(elem_classes="group-box"):
                question_box = gr.Textbox(label="💬 Ask a Question", placeholder="e.g. What is cloud computing?")
                ask_btn = gr.Button("🧠 Get Answer", variant="secondary")
                answer_box = gr.Textbox(label="🤖 Bot Answer", lines=10)

    # Connect logic
    process_btn.click(fn=process_pdf, inputs=pdf_file, outputs=status_box)
    ask_btn.click(fn=ask_question, inputs=question_box, outputs=answer_box)

demo.launch()


* Running on local URL:  http://127.0.0.1:7875
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://ec87e90c01a7245c81.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
